<a href="https://colab.research.google.com/github/KucharskiR/data-science/blob/main/LSTM_v_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback
from keras.initializers import TruncatedNormal
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# Replace 'your_file.csv' with the actual file path
file_path = '49f.csv'
file_labels = '49.csv'

# Specify the CSV file name
csv_file_name = 'my_data1.csv'

# Assuming there are three features in your data
num_features = 8
num_labels = 2
num_samples = 1980
timestepsPerSample = 120
epochs = 200
batch = 60

STEPS_PER_EPOCH = num_samples/batch

# Read the .csv file and create an array
data_strings = np.genfromtxt(file_path, delimiter=';')
labels_strings = np.genfromtxt(file_labels,delimiter=';')

# Convert from strings to float and int
X = data_strings.astype(float).reshape((num_samples,timestepsPerSample,num_features))
Y = labels_strings.astype(float).reshape((num_samples,num_labels))

# splitting the dataset 75% for training and 25% testing
# X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, shuffle=False)

# X_train = X_train.to_numpy()
x_train = X_train

lst = Sequential() # initializing model

# input layer and LSTM layer with 50 neurons
lst.add(LSTM(units=300, return_sequences=False, input_shape=(x_train.shape[1],x_train.shape[2])))
lst.add(Dense(100, activation='relu'))
lst.add(Dense(50, activation='relu'))
lst.add(Dense(25, activation='relu'))
# outpute layer with sigmoid activation
lst.add(Dense(num_labels, activation='sigmoid'))

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*100,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

optimizer = get_optimizer()

# defining loss function, optimizer, metrics and then compiling model
lst.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

# training the model on training dataset
# history = lst.fit(x_train, y_train, epochs=epochs, batch_size=batch,validation_split=0.2)
history = lst.fit(x_train, y_train, epochs=epochs, batch_size=batch)

# x_test = np.reshape(X_test, (X_test.shape[0],timestepsPerSample,X_test.shape[1]))
x_test = X_test

# predicting target attribute on testing dataset
predict = lst.predict(x_test)

# Set print options to suppress scientific notation
np.set_printoptions(suppress=True)

# Concatenate arrays
result = np.hstack((predict, y_test))
print(result)

test_results = lst.evaluate(x_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

# Create a DataFrame from the 2D array
df = pd.DataFrame(predict, columns=['Column1', 'Column2'])

# Export the DataFrame to CSV with semicolon as the delimiter and avoiding scientific notation
df.to_csv(csv_file_name, sep=';', index=False, float_format='%.0f')

# Read the CSV file into a DataFrame without header
df = pd.read_csv(csv_file_name, sep=';', header=None)

# Drop the first row containing data
df = df.iloc[1:]

# Save the modified DataFrame back to the CSV file without header
df.to_csv(csv_file_name, sep=';', index=False, header=False)

print(f'CSV file name: {csv_file_name}.')



Epoch 1/200
27/27 [==============================] - 19s 551ms/step - loss: 0.6620 - accuracy: 0.6174
Epoch 2/200
27/27 [==============================] - 15s 546ms/step - loss: 0.6177 - accuracy: 0.6610
Epoch 3/200
27/27 [==============================] - 15s 552ms/step - loss: 0.5861 - accuracy: 0.7052
Epoch 4/200
27/27 [==============================] - 15s 545ms/step - loss: 0.5905 - accuracy: 0.6856
Epoch 5/200
27/27 [==============================] - 15s 552ms/step - loss: 0.5555 - accuracy: 0.7109
Epoch 6/200
27/27 [==============================] - 16s 572ms/step - loss: 0.5525 - accuracy: 0.6995
Epoch 7/200
27/27 [==============================] - 17s 635ms/step - loss: 0.5238 - accuracy: 0.7336
Epoch 8/200
27/27 [==============================] - 16s 582ms/step - loss: 0.4916 - accuracy: 0.7639
Epoch 9/200
27/27 [==============================] - 15s 541ms/step - loss: 0.4805 - accuracy: 0.7734
Epoch 10/200
27/27 [==============================] - 15s 544ms/step - loss: 0.453